In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [3]:
G.register_tournaments()

In [4]:
G.register_matchups()

In [5]:
G.register_achievements()

In [16]:
T = G.tournaments["42884"]

In [6]:
tournaments = sorted(G.tournaments.values())

In [8]:
i = 34

In [14]:
T = tournaments[i]

In [17]:
print(f'{i} [{T.Id}] {T.name}')

34 [42884] Y2, Spring - Cathay and Nippon


In [ ]:
s = cibblbibbl.tournament.export.awards.plaintext.export(T, show_Ids=True)
print(s)

In [12]:
for A in T.achievements:
    if A["status"] == "proposed":
        A["status"] = "awarded"

In [13]:
i += 1

In [11]:
PO = "passed over, not awarded"

for A in T.achievements:
    if A["status"] == "proposed":
        print(A)
        print(A.subject.name)
        sys.stdout.flush()
        while True:
            answer = input("(a)warded/(p)assed over/(r)ejected (default: awarded)?").lower()
            if answer in ("", "a"):
                A["status"] = "awarded"
                break
            elif answer in ("p"):
                A["status"] = PO
                break
            elif answer in ("r"):
                A["status"] = "rejected"
                break

Child of Winter: Helmut Wulf (Star Player) (Fouls)
Helmut Wulf
(a)warded/(p)assed over/(r)ejected (default: awarded)?p
Child of Winter: Jessie Graff (Ninja Warriors) (Completions)
Jessie Graff
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Mosstail (Seven Samples of Decay) (SPP)
Mosstail
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Tutencal the Harvester (Guardians of the Lurking God) (Touchdowns)
Tutencal the Harvester
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Winter's Glory: Ninja Warriors
Ninja Warriors
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Ronin Lethal Strike (Secret Cult Of Shiamoto) (Blocking Scorer, Casualties)
Ronin Lethal Strike
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Winter's Glory: Lords of the Silent Death
Lords of the Silent Death
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Hazu (Secret Cult Of Shiamoto) (Blocking Thrower)
Hazu
(a)wa

In [ ]:
T.deadplayers()

In [27]:
not_dead = [
    [3893188, "STAR-Helmut Wulf"],
]

In [28]:
for matchId, playerId in not_dead:
    Ma = cibblbibbl.match.Match(matchId)
    Mu = Ma.matchup
    print(Mu)
    C = Mu.config
    for d0 in C["player"].values():
        for playerId1, d1 in d0.items():
            if playerId == playerId1:
                del d1["dead"]
                break
        else:
            continue
        break

Matchup('cibbl', '42748', 3, 872475, 872524)


In [25]:
admin_forfeits = [
    [["cibbl", "42041", 4, 855753, 856203], "856203"],
]

In [26]:
for matchup_KEY, teamId in admin_forfeits:
    Mu = Matchup(*matchup_KEY)
    d = Mu.config["team"][teamId]
    d["prestige"] = 0
    d["pts"] = 0
    d["r"] = "F"
    d["score"] = 0
    d["scorediff"] = -2
    d["tddiff"] = 0

In [28]:
excluded_matchups = [
    ["cibbl", "42239", 0, 863440, 863441],
    ["cibbl", "42239", 0, 863440, 864291],
    ["cibbl", "42239", 0, 863441, 864291],
]

In [29]:
for matchup_KEY in excluded_matchups:
    Mu = Matchup(*matchup_KEY)
    Mu.excluded = "yes"

In [31]:
admin_excludes = [
    [["cibbl", "44380", 1, 865063, 872475], "872475"],
    [["cibbl", "44380", 2, 865063, 874317], "874317"],
    [["cibbl", "44380", 4, 865063, 867030], "867030"],
    [["cibbl", "44380", 5, 865063, 869156], "869156"],
]

In [32]:
for matchup_KEY, teamId in admin_excludes:
    Mu = Matchup(*matchup_KEY)
    d = Mu.config["team"][teamId]
    d["prestige"] = 0
    d["pts"] = 0
    d["r"] = "-"
    d["score"] = 0
    d["scorediff"] = 0
    d["tddiff"] = 0

In [19]:
print(cibblbibbl.tournament.export.standings.plaintext.export(T))

 #                Name                      Roster                 Coach           Perf.    PTS   HTH   TDD   CAD   CTO
 1   Tower of the Necromancer         Necrarch              badenprinz            WLWDW      10     1     5    -2      
 2   Nachthafen Nightlords            Von Carstein          Danish_Dan            LWWDW      10     1     3    -6      
 3   Silver Pinnacle Renegades        Lahmian               Crook666              WWLWD      10     1     2     4      
 4   Strigos Elită                    Strigoi               Arcayn                LLWWD       7           1     5      
 5   Way of the Dragon                Blood Dragon          Nextflux              WWLLL       6          -1     8      
 6   Drakenhof Dance Macabre          Vampire               Bullroarer4           LLLLL       0         -10    -9      


In [13]:
print(cibblbibbl.tournament.export.prestiges.plaintext.export(T))

 #                Name                      Roster                 Coach           Perf.    GAM   CV   POS    P 
 1   Warhird Huscarls                 Norse Dwarf           Garion                DDWW                          
 2   Misty Isle Gators                Fimir                 Araznaroth            WDLW                          
 3   Albion Manticores                Four Nations          Mr_Fantastic          DDWL                          
 4   Slumbering Dragons               Albion                frogboy               DDLB                          
 5   Mount Thug Bonegrinders          Giant                 demigod               LDLF                          


In [11]:
T.retiredplayers()

{NormalPlayer('11748434'): {'mvp': 0,
  'spp': 0,
  'blocks': 0,
  'rush': 0,
  'td': 0,
  'turns': 0,
  'fouls': 0,
  'comp': 0,
  'pass': 0,
  'int': 0,
  'cas': 0,
  'perf': [('L', 3898310)],
  'team': Team(867030),
  'name': 'Ulric Paleorc',
  'type': 'J',
  'retired': True,
  'tournament': Tournament('cibbl', '42892')},
 NormalPlayer('11748435'): {'mvp': 0,
  'spp': 3,
  'blocks': 1,
  'rush': 0,
  'td': 1,
  'turns': 16,
  'fouls': 0,
  'comp': 0,
  'pass': 0,
  'int': 0,
  'cas': 0,
  'perf': [('L', 3898310)],
  'team': Team(867030),
  'name': 'Vincent Goldwing',
  'type': 'J',
  'retired': True,
  'tournament': Tournament('cibbl', '42892')},
 NormalPlayer('11748436'): {'mvp': 0,
  'spp': 3,
  'blocks': 1,
  'rush': 3,
  'td': 0,
  'turns': 11,
  'fouls': 0,
  'comp': 1,
  'pass': 5,
  'int': 0,
  'cas': 1,
  'perf': [('L', 3898310)],
  'team': Team(867030),
  'name': 'Ulric Clumsysnake',
  'type': 'J',
  'retired': True,
  'tournament': Tournament('cibbl', '42892')},
 NormalPla